# 4.3 質問応答のInstruction-Tuning

In [1]:
# ライブラリのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install 'transformers==4.51.3'
!pip install 'accelerate==0.23.0'
!pip install 'peft==0.15.2'
!pip install 'trl==0.19.0'
!pip install 'bitsandbytes==0.45.5'

  Using cached accelerate-0.23.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.8.1
    Uninstalling accelerate-1.8.1:
      Successfully uninstalled accelerate-1.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.19.0 requires accelerate>=1.4.0, but you have accelerate 0.23.0 which is incompatible.
  Using cached accelerate-1.8.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.8.1-py3-none-any.whl (365 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.23.0
    Uninstalling accelerate-0.23.0:
      Successfully uninstalled accelerate-0.23.0


In [2]:
# 警告抑制
import warnings
warnings.simplefilter("ignore")

In [3]:
# データセットの準備
# ※学習時間を短くするために学習データのサイズを縮小

import pandas as pd
from datasets import load_dataset, Dataset
# JCommonsenseQAをダウンロードし、データフレームに変換する
dataset_qa = load_dataset("shunk031/JGLUE", name="JCommonsenseQA")
train_df_qa = dataset_qa["train"].to_pandas()
test_df_qa = dataset_qa["validation"].to_pandas()
qa_df = pd.concat([train_df_qa, test_df_qa])

# llm-japanese-dataset-valnillaをダウンロードし、データフレーム変換する
dataset_llm_japanese = load_dataset("izumi-lab/llm-japanese-dataset-vanilla", revision="1.0.1")
train_df = dataset_llm_japanese["train"].to_pandas()
print(train_df.head(0))

# JCommonsenseQAのデータを除外する
qa_df["label_text"] = qa_df.apply(lambda x: x[f"choice{x['label']}"], axis=1)
print(qa_df.head(0))
mask = train_df["output"].isin(qa_df["label_text"]) & train_df["instruction"].isin(qa_df["question"])
train_df["is_JCommonsenseQA"] = mask
print(train_df.head(0))
train_df = train_df[~train_df["is_JCommonsenseQA"]]
print(train_df.head(0))
del train_df["is_JCommonsenseQA"]
print(f"除外したデータ数 : {sum(mask)}")

print("*"*300)

Empty DataFrame
Columns: [instruction, input, output]
Index: []
Empty DataFrame
Columns: [q_id, question, choice0, choice1, choice2, choice3, choice4, label, label_text]
Index: []
Empty DataFrame
Columns: [instruction, input, output, is_JCommonsenseQA]
Index: []
Empty DataFrame
Columns: [instruction, input, output, is_JCommonsenseQA]
Index: []
除外したデータ数 : 10059
************************************************************************************************************************************************************************************************************************************************************************************************************


In [4]:
#　InstructionTuning用のデータセットを作成
# JCommonsenseQAデータセットの整備
qa_instruction_text = "質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。"
train_df_qa["instruction"] = qa_instruction_text
print(train_df_qa.head())
train_df_qa["input"] = train_df_qa.apply(lambda x: f"質問:\n{x['question']}\n選択肢:\n0.{x['choice0']} 1.{x['choice1']} 2.{x['choice2']} 3.{x['choice3']} 4.{x['choice4']}", axis=1)
train_df_qa["output"] = train_df_qa["label"].astype(str)
train_df_qa = train_df_qa[["instruction", "input", "output"]]

print(train_df_qa.head())
print("*"*300)

   q_id                                     question choice0 choice1 choice2  \
0     0          主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？      世界     写真集      絵本   
1     1  未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？     浮浪者     保護者    お坊さん   
2     2                              数字の１を表すときに使う体は？       胸      肉球      背中   
3     3                        火を起こすとあらわれるもくもくするものは？    歯の変色      ガス      中毒   
4     4                                  大型商業施設といえば？    アパート   レジデンス   ドレッサー   

  choice3 choice4  label                                        instruction  
0      論文      図鑑      2  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...  
1     宗教者     預言者      1  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...  
2    人差し指      親指      3  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...  
3      爆発       煙      4  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...  
4    デパート    衣料品店      3  質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択...  
                                         instructio

In [5]:
# 短時間で学習可能なサイズにサンプリングする
total_sample_size = 30000
sample_size = total_sample_size - len(train_df_qa)
train_df = train_df.sample(n=sample_size, random_state=42)
print(train_df.head())
print("*"*300)

                instruction                   input  \
1729784  入力されたワードを説明してください。          寝癖 (クリープハイプの曲)   
1158591  入力されたワードを説明してください。               マーク・ランドール   
2123868  入力されたワードを説明してください。                   小笠原則普   
1552823  入力されたワードを説明してください。  BOAT RACEライブ 〜勝利へのターン〜   
1225719  入力されたワードを説明してください。                      挙党   

                                                    output  
1729784  「寝癖」（ねぐせ）は、クリープハイプのメジャー4枚目のシングル。2014年5月14日にユニバ...  
1158591  マーク・レナード・ランドール（Mark Leonard Randall, 1989年9月28...  
2123868  小笠原 則普（おがさわら のりひろ、寛保3年（1743年） - 没年不詳）は、江戸時代後期の...  
1552823  BOAT RACE ライブ（ぼーとれーすらいぶ）は、BSフジで2011年4月3日から生放送さ...  
1225719  挙党（きょとう）とは、政党内で対立している勢力を包含し、反主流派を作らず党を挙げて物事に取り...  
************************************************************************************************************************************************************************************************************************************************************************************************************


In [6]:
# train_dfと結合
train_df = pd.concat([train_df, train_df_qa], ignore_index=True)
print(train_df.head())
print("*"*300)

          instruction                   input  \
0  入力されたワードを説明してください。          寝癖 (クリープハイプの曲)   
1  入力されたワードを説明してください。               マーク・ランドール   
2  入力されたワードを説明してください。                   小笠原則普   
3  入力されたワードを説明してください。  BOAT RACEライブ 〜勝利へのターン〜   
4  入力されたワードを説明してください。                      挙党   

                                              output  
0  「寝癖」（ねぐせ）は、クリープハイプのメジャー4枚目のシングル。2014年5月14日にユニバ...  
1  マーク・レナード・ランドール（Mark Leonard Randall, 1989年9月28...  
2  小笠原 則普（おがさわら のりひろ、寛保3年（1743年） - 没年不詳）は、江戸時代後期の...  
3  BOAT RACE ライブ（ぼーとれーすらいぶ）は、BSフジで2011年4月3日から生放送さ...  
4  挙党（きょとう）とは、政党内で対立している勢力を包含し、反主流派を作らず党を挙げて物事に取り...  
************************************************************************************************************************************************************************************************************************************************************************************************************


In [7]:
# datasetクラスに変換
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(train_size=0.9, seed=42)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 27000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3000
    })
})


In [8]:
# 環境変数の設定
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["LOCAL_RANK"] = "0"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

In [9]:
# モデル設定
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 量子化
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# トークナイザ, モデルの読み込み
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-3.1-1.8b-instruct4", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-3.1-1.8b-instruct4", quantization_config=bnb_config, device_map="cuda:0")
print(model)

[2025-06-24 11:43:55,692] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(99584, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=7168, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=7168, bias=False)
          (down_proj): Linear8bitLt(in_features=7168, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2

In [10]:
# 学習前の出力確認用
from transformers import pipeline
# パイプラインの構築
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
prompt = """### 指示: 以下の質問に回答してください。 ### 入力: 日本一高い山は？ ### 回答: """

generate_text = qa_pipeline(prompt, max_length=100, num_return_sequences=1, temperature=0.6)[0]["generated_text"]
print(generate_text)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
富士山


In [11]:
# LoRA設定
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

peft_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"],
    lora_dropout=0.05,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)

In [13]:
# パラメーター確認
if hasattr(model, 'print_trainable_parameters'):
    print("\n=== PEFT Model Information ===")
    print(model.print_trainable_parameters())


=== PEFT Model Information ===
trainable params: 67,633,152 || all params: 1,935,247,360 || trainable%: 3.4948
None


In [14]:
# TrainingArgumentsの設定
training_arguments = TrainingArguments(
    output_dir="./llm-jp-3-1-1.8b-instruct4-8bit-lora",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=5000,
)

In [15]:
# DataCollatorの設定
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"### 指示:\n{example['instruction'][i]}\n\n### 入力:\n{example['input'][i]}\n\n### 回答:\n{example['output'][i]}"
        output_texts.append(text)
    return output_texts

collator = DataCollatorForCompletionOnlyLM(response_template="回答:\n", tokenizer=tokenizer)

In [16]:
# モデルの学習
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_arguments,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)
trainer.train()

Applying formatting function to train dataset:   0%|          | 0/27000 [00:00<?, ? examples/s]

Applying formatting function to train dataset:   0%|          | 0/27000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/27000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/27000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/27000 [00:00<?, ? examples/s]

...

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: arekunoimar (arekunoimar-deepspeed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[rank0]:[W624 11:44:08.290038653 reducer.cpp:1400] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())


Step,Training Loss,Validation Loss
5000,0.929500,0.966043
10000,0.729700,0.940104
15000,0.942900,0.928775
20000,0.875600,0.913986
25000,0.888600,0.918031


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=27000, training_loss=0.9539450028737386, metrics={'train_runtime': 4310.0832, 'train_samples_per_second': 6.264, 'train_steps_per_second': 6.264, 'total_flos': 2.635549006574387e+16, 'train_loss': 0.9539450028737386})

In [20]:
trainer.save_model() # モデルの保存
trainer.save_state() # メトリクスの保存